In [1]:
import pandas as pd
import sqlite3 as lite
import ast
import json
import re
import time
import pickle
import os

In [2]:
# langs = ['C', 'PHP','C++', 'JavaScript', 'Python', 'Ruby', 'Java', 
#  'Shell', 'Go', 'TypeScript', 'Objective-C', 'SQL', 'C#', 'Perl', 
#  'Batchfile', 'CoffeeScript', 'Scala', 'PowerShell', 'Haskell', 'Lua',
#  'Rust', 'Swift', 'R', 'Matlab', 'Erlang' ]
langs = ["PHP", "C", "JavaScript", "Python", "Java", "TypeScript", "C++", "Go", "Ruby"]

remove_langs = ['unknown', 'Markdown', 'None', 'HTML', 'TeX', 'CSS', 'Jupyter Notebook']
remove_cwe = ['NVD-CWE-noinfo', 'NVD-CWE-Other']

cwe_top_25 = [
    "CWE-79",  # Improper Neutralization of Input During Web Page Generation ('Cross-site Scripting')
    "CWE-787",  # Out-of-bounds Write
    "CWE-89",  # Improper Neutralization of Special Elements used in an SQL Command ('SQL Injection')
    "CWE-352",  # Cross-Site Request Forgery (CSRF)
    "CWE-22",  # Improper Limitation of a Pathname to a Restricted Directory ('Path Traversal')
    "CWE-125",  # Out-of-bounds Read
    "CWE-78",  # Improper Neutralization of Special Elements used in an OS Command ('OS Command Injection')
    "CWE-416",  # Use After Free
    "CWE-862",  # Missing Authorization
    "CWE-434",  # Unrestricted Upload of File with Dangerous Type
    "CWE-94",  # Improper Control of Generation of Code ('Code Injection')
    "CWE-20",  # Improper Input Validation
    "CWE-77",  # Improper Neutralization of Special Elements used in a Command ('Command Injection')
    "CWE-287",  # Improper Authentication
    "CWE-269",  # Improper Privilege Management
    "CWE-502",  # Deserialization of Untrusted Data
    "CWE-200",  # Exposure of Sensitive Information to an Unauthorized Actor
    "CWE-863",  # Incorrect Authorization
    "CWE-918",  # Server-Side Request Forgery (SSRF)
    "CWE-119",  # Improper Restriction of Operations within the Bounds of a Memory Buffer
    "CWE-476",  # NULL Pointer Dereference
    "CWE-798",  # Use of Hard-coded Credentials
    "CWE-190",  # Integer Overflow or Wraparound
    "CWE-400",  # Uncontrolled Resource Consumption
    "CWE-306"   # Missing Authentication for Critical Function
]
chosen_cwes = ['CWE-20', 'CWE-287', 'CWE-400', 'CWE-668', 'CWE-74']


In [3]:
def create_connection(db_file):
    """
    create a connection to sqlite3 database
    """
    conn = None
    try:
        conn = lite.connect(db_file, timeout=10)  # connection via sqlite3
    except Error as e:
        print(e)
    return conn

conn = create_connection('/home/keisuke/code/llm-code-vuln/dataset/CVEfixes_v1.0.8/Data/DB.db')

## XML Tree

In [4]:
import xml.etree.ElementTree as ET

# Parse the XML file
tree = ET.parse("./dataset/cwe.xml")
root = tree.getroot()
ns = {"cwe": "http://cwe.mitre.org/cwe-7"} # Define namespace
cwe_dict = {weakness.get("ID"): weakness for weakness in root.findall(".//cwe:Weakness", ns)}

# Function that tells type
def get_type(cwe_id:str):
    weakness = cwe_dict.get(cwe_id)
    abstraction = weakness.get("Abstraction")
    return abstraction

# Function that find parent based on specified id
def find_parents(cwe_id:str):    
    result = []
    
    current_cwe_id = cwe_id
    while current_cwe_id:    
        # Find the Weakness element
        weakness = cwe_dict.get(current_cwe_id)
        if not weakness:
            return None  # If CWE is not found, stop the search

        abstraction = weakness.get("Abstraction")
        if abstraction == "Pillar": break
        
        if current_cwe_id not in result and cwe_id != current_cwe_id:
            result.append("CWE-" +current_cwe_id)
        
        # Check if the weakness abstraction is "Pillar"
        
#         if "CWE-" + current_cwe_id in cwe_top_25: break

        # Find the Parent CWE (ChildOf relation)
        related_weaknesses = weakness.find("cwe:Related_Weaknesses", ns)
        if related_weaknesses is not None:
            for related in related_weaknesses.findall("cwe:Related_Weakness", ns):
                if related.get("Nature") == "ChildOf":
                    current_cwe_id = related.get("CWE_ID")
        else: break
    
    return result

def find_parents_dict(cwe_id:str):    
    result = {"Pillar": [], "Class": [], "Base": [], "Variant": []}
    
    current_cwe_id = cwe_id
    while current_cwe_id:    
        # Find the Weakness element
        weakness = cwe_dict.get(current_cwe_id)
        if not weakness:
            return None  # If CWE is not found, stop the search

        abstraction = weakness.get("Abstraction")
        if (result.get(abstraction, None)) is not None:
            result[abstraction].append('CWE-'+current_cwe_id)
        
        # Check if the weakness abstraction is "Pillar"
        if abstraction == "Pillar": break

        # Find the Parent CWE (ChildOf relation)
        related_weaknesses = weakness.find("cwe:Related_Weaknesses", ns)
        if related_weaknesses is not None:
            for related in related_weaknesses.findall("cwe:Related_Weakness", ns):
                if related.get("Nature") == "ChildOf":
                    current_cwe_id = related.get("CWE_ID")
        else: break
    
    return result

## Processing data

In [5]:
def pre_processing(df):
    # keep only the specified programming languages
    df = df[df['programming_language'].isin(langs)]

    # code diff: both add & del should exist
    df['diff_added'] = df.apply(lambda row: ast.literal_eval(row.diff_parsed)['added'], axis=1)
    df['diff_deleted'] = df.apply(lambda row: ast.literal_eval(row.diff_parsed)['deleted'], axis=1)
    df = df[df['diff_added'].apply(bool) & df['diff_deleted'].apply(bool)]
    df = df.reset_index(drop=True)
    df = df.drop(columns=['diff_parsed'])

    # cve description type str -> arr
    def parse_py_literal(text):
        if not isinstance(text, str):
            return text
        try:
            return ast.literal_eval(text)
        except (SyntaxError, ValueError):
            return None

    df['cve_description'] = df['cve_description'].apply(parse_py_literal)

    # code before and after
    df = df[df['vuln_code'].notna()]
    df = df[df['vuln_code'] != 'None']
    df = df[df['non_vuln_code'].notna()]
    df = df[df['non_vuln_code'] != 'None']

    # remove rows where number of lines in the code is below 30
    for col in ['vuln_code', 'non_vuln_code']:
        df[f'{col}_num_lines'] = df[col].apply(
            lambda x: x.count('\n') + 1 if isinstance(x, str) else 0
        )
        df = df[df[f'{col}_num_lines'] >= 30]

    # remove empty list in diff_deleted
    df = df[df['diff_deleted'].apply(lambda x: isinstance(x, list) and len(x) > 0)]

    # token_count should be num
    df['token_count'] = df['token_count'].apply(parse_py_literal)
    df = df.dropna(subset=['token_count'])
    df['token_count'] = pd.to_numeric(df['token_count'])

    # drop the other CWEs
    df = df[~df["cwe_id"].isin(remove_cwe)]

    # Add Cluss column (Parent CWE)
    cwe_uniques = df['cwe_id'].unique()
    for cwe in cwe_uniques:
        parents = find_parents_dict(cwe.split('-')[1])
        if parents is None or len(parents['Class']) == 0:
            continue
        df.loc[df['cwe_id'] == cwe, 'class'] = parents['Class'][-1]

    df = df.dropna()
    return df

def pick_samples(df):
    sample_size = 10
    vuln_selected = []
    non_vuln_selected = []

    for lang in langs:
        # Vuln samples
        for cwe in chosen_cwes:
            filtered = df[
                (df['programming_language'] == lang) &
                (df['class'] == cwe) &
                (df['token_count'] <= 8000)
            ]
            total_available = len(filtered)
            count = 0

            if len(filtered) == 0:
                print(f'! {lang} {cwe} No samples available')
                samples = None  # Indicate there's no data
            elif len(filtered) < sample_size:
                print(f'- {lang} {cwe} {len(filtered)} Not enough samples')
                samples = filtered  # Use all available samples
                count = len(filtered)
            else:
                print(f'+ {lang} {cwe} {len(filtered)}')
                samples = filtered.sample(n=sample_size, random_state=123)
                count = len(samples)

            if samples is not None:
                df = df.drop(samples.index)
                vuln_selected.append(samples)

            percentage_used = (count / total_available) * 100 if total_available > 0 else 0
            print(f'{lang}:{cwe}: {count}/{total_available} ({percentage_used:.2f}%)')
        
        # Non-vuln samples
        filtered = df[
            (df['programming_language'] == lang) &
            (df['token_count'] <= 15000)
        ]
        samples = filtered.sample(n=sample_size, random_state=123)
        non_vuln_selected.append(samples)
        df = df.drop(samples.index)

    # Combine all selected samples into one DataFrame
    final_vuln_samples = pd.concat(vuln_selected, ignore_index=True) if vuln_selected else pd.DataFrame()
    final_non_vuln_samples = pd.concat(non_vuln_selected, ignore_index=True) if non_vuln_selected else pd.DataFrame()

    # Display final sampled DataFrame
    print(f'final vuln sample size: {len(final_vuln_samples)}')
    print(f'final non-vulns sample size: {len(final_non_vuln_samples)}')

    return final_vuln_samples, final_non_vuln_samples, df


In [6]:
query = f"""
SELECT
    file_change.file_change_id,
    file_change.programming_language,
    cwe.cwe_id,
    cwe.cwe_name,
    file_change.code_after AS non_vuln_code,
    file_change.code_before AS vuln_code,
    cwe.description AS cwe_description,
    file_change.diff_parsed,
    cve.description AS cve_description,
    file_change.token_count
FROM file_change
    INNER JOIN fixes
        ON file_change.hash = fixes.hash
    INNER JOIN cve
        ON fixes.cve_id = cve.cve_id
    INNER JOIN cwe_classification
        ON cve.cve_id = cwe_classification.cve_id
    INNER JOIN cwe
        ON cwe_classification.cwe_id = cwe.cwe_id
WHERE
    file_change.programming_language IS NOT NULL
    AND cwe.cwe_id IS NOT NULL
    AND cwe.cwe_name IS NOT NULL
    AND file_change.code_before IS NOT NULL
    AND cwe.description IS NOT NULL
    AND file_change.diff_parsed IS NOT NULL
    AND cve.description IS NOT NULL
    AND file_change.token_count IS NOT NULL;
"""
df = pd.read_sql(query, con=conn)
df = pre_processing(df)
vuln, non_vuln, df = pick_samples(df)

/tmp/ipykernel_1100786/1842481859.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['diff_added'] = df.apply(lambda row: ast.literal_eval(row.diff_parsed)['added'], axis=1)
/tmp/ipykernel_1100786/1842481859.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['diff_deleted'] = df.apply(lambda row: ast.literal_eval(row.diff_parsed)['deleted'], axis=1)


+ PHP CWE-20 347
PHP:CWE-20: 10/347 (2.88%)
+ PHP CWE-287 88
PHP:CWE-287: 10/88 (11.36%)
+ PHP CWE-400 42
PHP:CWE-400: 10/42 (23.81%)
+ PHP CWE-668 462
PHP:CWE-668: 10/462 (2.16%)
+ PHP CWE-74 3521
PHP:CWE-74: 10/3521 (0.28%)
+ C CWE-20 1769
C:CWE-20: 10/1769 (0.57%)
+ C CWE-287 51
C:CWE-287: 10/51 (19.61%)
+ C CWE-400 84
C:CWE-400: 10/84 (11.90%)
+ C CWE-668 118
C:CWE-668: 10/118 (8.47%)
+ C CWE-74 44
C:CWE-74: 10/44 (22.73%)
+ JavaScript CWE-20 60
JavaScript:CWE-20: 10/60 (16.67%)
+ JavaScript CWE-287 24
JavaScript:CWE-287: 10/24 (41.67%)
+ JavaScript CWE-400 78
JavaScript:CWE-400: 10/78 (12.82%)
+ JavaScript CWE-668 84
JavaScript:CWE-668: 10/84 (11.90%)
+ JavaScript CWE-74 754
JavaScript:CWE-74: 10/754 (1.33%)
+ Python CWE-20 112
Python:CWE-20: 10/112 (8.93%)
+ Python CWE-287 66
Python:CWE-287: 10/66 (15.15%)
+ Python CWE-400 100
Python:CWE-400: 10/100 (10.00%)
+ Python CWE-668 194
Python:CWE-668: 10/194 (5.15%)
+ Python CWE-74 234
Python:CWE-74: 10/234 (4.27%)
+ Java CWE-20 47
Java

## CWE Find parents

In [7]:
cwe_uniques = df['cwe_id'].unique()

cwe_parents = {}
for cwe in cwe_uniques:
    parents = find_parents_dict(cwe.split('-')[1])
    if parents is None or len(parents['Class'])==0 : continue
    df.loc[df['cwe_id'] == cwe, 'class'] = parents['Class'][-1]

df_filtered = df[(df['class'].notna())]

# Count occurrences of each (cwe_id, programming_language)
df_counts = df_filtered.groupby(['class', 'programming_language']).size().reset_index(name='count')

# Filter for CWE IDs that have at least 10 samples per language
df_valid = df_counts[df_counts['count'] >= 10]

# Find CWE IDs that appear in all the languages
valid_cwes = df_valid.groupby('class')['programming_language'].nunique()
cwe_with_all_langs = valid_cwes[valid_cwes == len(langs)].index

# Get final dataset with only these CWE IDs
final_df = df_valid[df_valid['class'].isin(cwe_with_all_langs)]

# Display the results
# print(final_df)

chosen_cwe = final_df['class'].unique()
print(chosen_cwe)

['CWE-20' 'CWE-287' 'CWE-400' 'CWE-668']


In [8]:
df

,file_change_id,programming_language,cwe_id,cwe_name,non_vuln_code,vuln_code,cwe_description,cve_description,token_count,diff_added,diff_deleted,vuln_code_num_lines,non_vuln_code_num_lines,class
0,41461181100456,C,CWE-252,Unchecked Return Value,/* Map in a shared object's segments from the ...,/* Map in a shared object's segments from the ...,The product does not check the return value fr...,"[{'lang': 'en', 'value': 'manual/search.texi i...",6592.0,"[(152, const char *const start = name;), (15...","[(158, /* $ORIGIN is not expanded for SU...",1449,1467,CWE-754
1,258466377194482,C,CWE-252,Unchecked Return Value,"/*\n * Copyright (c) 1983, 1988, 1993\n *\tThe...","/*\n * Copyright (c) 1983, 1988, 1993\n *\tThe...",The product does not check the return value fr...,"[{'lang': 'en', 'value': 'manual/search.texi i...",1240.0,"[(180, \t\t/* Append a newline if necessary. ...","[(180, \t\t++v;), (181, \t\tv->iov_base = (cha...",356,360,CWE-754
2,23846676397498,C,CWE-252,Unchecked Return Value,/* Extended regular expression matching and se...,/* Extended regular expression matching and se...,The product does not check the return value fr...,"[{'lang': 'en', 'value': 'manual/search.texi i...",14578.0,"[(239, #ifndef emacs), (240, /* How many chara...","[(167, /* How many characters in the character...",5881,5878,CWE-754
3,141694920932084,C,CWE-252,Unchecked Return Value,/* Reentrant function to return the current lo...,/* Reentrant function to return the current lo...,The product does not check the return value fr...,"[{'lang': 'en', 'value': 'manual/search.texi i...",195.0,"[(43, /* Get name of tty connected to fd 0. ...","[(43, {), (44, int d = __open (""/dev/tty...",89,89,CWE-754
6,138267799803328,C,CWE-20,Improper Input Validation,/* -*- mode: c; c-basic-offset: 4; indent-tabs...,/* -*- mode: c; c-basic-offset: 4; indent-tabs...,"The product receives input or data, but it doe...","[{'lang': 'en', 'value': 'schpw.c in the kpass...",2230.0,"[(55, goto bailout;), (70, got...","[(55, goto chpwfail;), (70, go...",487,487,CWE-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27260,233098165980480,Python,CWE-346,Origin Validation Error,"from flask import blueprints, request, jsonify...","from flask import blueprints, request, jsonify...",The product does not properly verify that the ...,"[{'lang': 'en', 'value': 'A CORS misconfigurat...",371.0,"[(2, from werkzeug.utils import secure_filenam...","[(21, manager.create_project(project_name)...",54,63,CWE-346
27261,233098165980480,Python,CWE-79,Improper Neutralization of Input During Web Pa...,"from flask import blueprints, request, jsonify...","from flask import blueprints, request, jsonify...",The product does not neutralize or incorrectly...,"[{'lang': 'en', 'value': 'A stored Cross-Site ...",371.0,"[(2, from werkzeug.utils import secure_filenam...","[(21, manager.create_project(project_name)...",54,63,CWE-74
27264,168198219170456,Python,CWE-22,Improper Limitation of a Pathname to a Restric...,import json\nimport os\nfrom datetime import d...,import json\nimport os\nfrom datetime import d...,The product uses external input to construct a...,"[{'lang': 'en', 'value': 'A directory traversa...",1284.0,"[(177, )]","[(177, ), (178, def get_project_files(self...",201,177,CWE-668
27265,168198219170456,Python,CWE-346,Origin Validation Error,import json\nimport os\nfrom datetime import d...,import json\nimport os\nfrom datetime import d...,The product does not properly verify that the ...,"[{'lang': 'en', 'value': 'A CORS misconfigurat...",1284.0,"[(177, )]","[(177, ), (178, def get_project_files(self...",201,177,CWE-346


In [9]:
import ollama

In [15]:
from langchain_community.chat_models.ollama import ChatOllama

# Initialize the ChatOllama model
chat_model = ChatOllama()

# Your input text
text = "Your input text here."

# Get the number of tokens in the text
num_tokens = chat_model.get_num_tokens(text)

print(f"The number of tokens in the input text is: {num_tokens}")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

The number of tokens in the input text is: 5


In [19]:
import tiktoken

# Load the encoding for the model you're using
encoding = tiktoken.encoding_for_model("gpt-4o")

# Your input text
text = "Your input text here."

# Encode the text to get the list of tokens
tokens = encoding.encode(text)

# Count the number of tokens
num_tokens = len(tokens)

print(f"The number of tokens in the input text is: {num_tokens}")

The number of tokens in the input text is: 5


In [20]:
with open('./dataset/test_pickles/test_vuln.pkl', 'rb') as file:
    tmp = pickle.load(file)

In [21]:
tmp

,file_change_id,programming_language,cwe_id,cwe_name,non_vuln_code,vuln_code,cwe_description,cve_description,token_count,diff_added,diff_deleted,vuln_code_num_lines,non_vuln_code_num_lines,class
0,204963180360770,PHP,CWE-20,Improper Input Validation,<?php\n\n#####################################...,<?php\n\n#####################################...,"The product receives input or data, but it doe...","[{'lang': 'en', 'value': 'Exponent CMS before ...",5918.0,"[(77, /*if ($quantity < $c...","[(77, /*if ($quantity < $c...",1268,1268,CWE-20
1,133489664160898,PHP,CWE-20,Improper Input Validation,<?php\n\n/*\n * Wolf CMS - Content Management ...,<?php\n\n/*\n * Wolf CMS - Content Management ...,"The product receives input or data, but it doe...","[{'lang': 'en', 'value': 'Wolf CMS before 0.8....",4149.0,"[(30, *), (58, static function htmlContex...","[(30, *), (97, 'dir' => $this->pa...",717,739,CWE-20
2,268931637252246,PHP,CWE-20,Improper Input Validation,<?php\n/*\n Licensed to the Apache Software F...,<?php\n/*\n Licensed to the Apache Software F...,"The product receives input or data, but it doe...","[{'lang': 'en', 'value': 'The Privileges porti...",13432.0,"[(72, /// HTTP headers do not need to be passe...","[(72, /// HTTP headers do not need to be passe...",3217,3509,CWE-20
3,223579129914434,PHP,CWE-73,External Control of File Name or Path,<?php\n\ndeclare(strict_types=1);\n\n/**\n * T...,<?php\n\ndeclare(strict_types=1);\n\n/**\n * T...,The product allows user input to control or in...,"[{'lang': 'en', 'value': 'External Control of ...",561.0,"[(14, * @version 3.0.0.23)]","[(14, * @version 3.0.0.22)]",245,245,CWE-20
4,223691666203064,PHP,CWE-20,Improper Input Validation,<?php\n/**************************************...,<?php\n/**************************************...,"The product receives input or data, but it doe...","[{'lang': 'en', 'value': 'Incomplete blacklist...",97.0,"[(66, Yii::app()->basePath,), (67,...","[(66, Yii::app()->basePath...",75,75,CWE-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,105677552550412,Ruby,CWE-74,Improper Neutralization of Special Elements in...,# frozen_string_literal: true\n\nrequire_relat...,# frozen_string_literal: true\n\nrequire_relat...,The product constructs all or part of a comman...,"[{'lang': 'en', 'value': 'A vulnerability was ...",1519.0,"[(80, it 'excludes CAS if set to 0' do), (...","[(96, it 'returns incorporates CAS when pa...",140,243,CWE-74
446,273147407334227,Ruby,CWE-89,Improper Neutralization of Special Elements us...,#\n#= MailFolder\n#\n#Original by:: Sysphoni...,#\n#= MailFolder\n#\n#Original by:: Sysphoni...,The product constructs all or part of an SQL c...,"[{'lang': 'en', 'value': 'Multiple SQL injecti...",1413.0,"[(125, SqlHelper.validate_token([mail_acco...","[(154, con << ""(user_id=#{user_id})""), (15...",497,499,CWE-74
447,72971686250412,Ruby,CWE-79,Improper Neutralization of Input During Web Pa...,"# frozen_string_literal: true\n\nrequire ""rail...","# frozen_string_literal: true\n\nrequire ""rail...",The product does not neutralize or incorrectly...,"[{'lang': 'en', 'value': 'In Publify, versions...",929.0,"[(163, let(:original_html) { '<a href=""htt...","[(171, expect(nofollowify_links('<a href...",230,244,CWE-74
448,214657170652761,Ruby,CWE-89,Improper Neutralization of Special Elements us...,#\n#= WorkflowsController\n#\n#Original by:: ...,#\n#= WorkflowsController\n#\n#Original by:: ...,The product constructs all or part of an SQL c...,"[{'lang': 'en', 'value': 'Multiple SQL injecti...",585.0,"[(6, #Copyright:: Copyright (c) 2007-2015 MORI...","[(6, #Copyright:: Copyright (c) 2007-2011 MORI...",150,156,CWE-74
